In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
import pickle


data = pd.read_csv('PS_20174392719_1491204439457_log.csv')
data = data.drop('type', axis=1)
data = data.drop('nameOrig', axis=1)
data = data.drop('nameDest', axis=1)
X = data.drop('isFraud', axis=1)
y = data['isFraud']

In [2]:
X.head();

print(X);

         step      amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
0           1     9839.64      170136.00       160296.36            0.00   
1           1     1864.28       21249.00        19384.72            0.00   
2           1      181.00         181.00            0.00            0.00   
3           1      181.00         181.00            0.00        21182.00   
4           1    11668.14       41554.00        29885.86            0.00   
...       ...         ...            ...             ...             ...   
6362615   743   339682.13      339682.13            0.00            0.00   
6362616   743  6311409.28     6311409.28            0.00            0.00   
6362617   743  6311409.28     6311409.28            0.00        68488.84   
6362618   743   850002.52      850002.52            0.00            0.00   
6362619   743   850002.52      850002.52            0.00      6510099.11   

         newbalanceDest  isFlaggedFraud  
0                  0.00               0  
1  

In [3]:
print(y)

0          0
1          0
2          1
3          1
4          0
          ..
6362615    1
6362616    1
6362617    1
6362618    1
6362619    1
Name: isFraud, Length: 6362620, dtype: int64


In [4]:
X=X.dropna()
y=y.dropna()
print(X)
print(y)

         step      amount  oldbalanceOrg  newbalanceOrig  oldbalanceDest  \
0           1     9839.64      170136.00       160296.36            0.00   
1           1     1864.28       21249.00        19384.72            0.00   
2           1      181.00         181.00            0.00            0.00   
3           1      181.00         181.00            0.00        21182.00   
4           1    11668.14       41554.00        29885.86            0.00   
...       ...         ...            ...             ...             ...   
6362615   743   339682.13      339682.13            0.00            0.00   
6362616   743  6311409.28     6311409.28            0.00            0.00   
6362617   743  6311409.28     6311409.28            0.00        68488.84   
6362618   743   850002.52      850002.52            0.00            0.00   
6362619   743   850002.52      850002.52            0.00      6510099.11   

         newbalanceDest  isFlaggedFraud  
0                  0.00               0  
1  

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=39)

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled)

[[-1.60450119  0.36329091 -0.23692319 ... -0.32380565 -0.33351925
  -0.00165845]
 [ 1.05142395 -0.12278162 -0.27681958 ... -0.32380565 -0.30464927
  -0.00165845]
 [-0.43111627 -0.28982184 -0.18452987 ... -0.32380565 -0.33351925
  -0.00165845]
 ...
 [ 1.12871278 -0.26767692 -0.2887363  ... -0.26979185 -0.28862541
  -0.00165845]
 [ 0.54553345 -0.29709705 -0.28825795 ... -0.32380565 -0.33351925
  -0.00165845]
 [-1.64665873  0.10208128 -0.28883806 ... -0.17831763 -0.03573017
  -0.00165845]]


In [7]:
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)
y_pred_rf = random_forest_model.predict(X_test)

In [8]:
# Save the Random Forest model
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(random_forest_model, file)

In [9]:
neural_network_model = Sequential()
neural_network_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
neural_network_model.add(Dense(1, activation='sigmoid'))
neural_network_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
neural_network_model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)
pred_x=neural_network_model.predict(X_test);
y_pred_nn=np.argmax(pred_x,axis=1);

Epoch 1/5
79533/79533 [==============================] - 183s 2ms/step - loss: 139.7190 - accuracy: 0.9938
Epoch 2/5
79533/79533 [==============================] - 172s 2ms/step - loss: 57.6170 - accuracy: 0.9956
Epoch 3/5
79533/79533 [==============================] - 148s 2ms/step - loss: 50.7067 - accuracy: 0.9963
Epoch 4/5
79533/79533 [==============================] - 149s 2ms/step - loss: 61.0992 - accuracy: 0.9966
Epoch 5/5
39767/39767 [==============================] - 58s 1ms/step


In [10]:
# Save the Neural Network model
# with open('neural_network_model.pkl', 'wb') as file:
#     pickle.dump(neural_network_model, file)
neural_network_model.save('neural_network_model.h5')

In [11]:
y_pred_combined = np.round((y_pred_rf + y_pred_nn) / 2)

In [12]:
accuracy = accuracy_score(y_test, y_pred_combined)
conf_matrix = confusion_matrix(y_test, y_pred_combined)
classification_rep = classification_report(y_test, y_pred_combined)

c:\Users\vishw\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vishw\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\vishw\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [13]:
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{classification_rep}')


Accuracy: 0.9987780191179105
Confusion Matrix:
[[1270969       0]
 [   1555       0]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270969
           1       0.00      0.00      0.00      1555

    accuracy                           1.00   1272524
   macro avg       0.50      0.50      0.50   1272524
weighted avg       1.00      1.00      1.00   1272524

